In [7]:
# Imports

import tkinter as tk, json, ctypes
from tkinter import ttk

dpi = ctypes.windll.shcore.SetProcessDpiAwareness(True)

In [8]:
# Lists

characters = json.load(open('Character_Offsets.json', 'r'))
classes = json.load(open('Class_Offsets.json', 'r'))
items = json.load(open('Item_Offsets.json', 'r'))

char_sel = list(characters['Name'])
class_sel = list(classes['Name'])
item_sel = list(items['Name'])

In [9]:
# Description for info tab

desc = {
    "intro": "Welcome to the Fire Emblem Radiant Dawn Code Creator! This tool will allow you to easily create Gecko Codes to change and add a variety of data to your game. Please see each section below for more details.",
    "Misc Information": "Text Input Fields - Forge Name has a max character count of 26. All other input fields are for numeric input with a max value of 255.\nDropdowns - All dropdowns are pre-populated and can only take values from the list provided. You can type in the name of the character, class or item, but it needs to be exact or the code will not find it.",
    "Keybind Activation Tab": "Allows users to select a controller type and configure keybindings required to activate the code. This tab only works in conjunction with the character tab. The codes generated from Class and Items are codes that need to be always on.",
    "Character Tab": "Lets users select a character and configure their items, including forge names, uses, and various attributes. Make sure to pair with input in the keybinds tab to activate based on custom button pairing! If no keybinds are selected, the code defaults to an always on status and will repeatedly write to your character.",
    "Class Tab": "Allows users to select a class and configure max weapon ranks and stats.",
    "Items Tab": "Allows users to select an item and configure its miscellaneous data and equip bonuses."
}

In [10]:
# UDF

def get_char_code(data, kb):
    output = []
    output.append(kb)
    char = data['character']
    if not char:
        return "No character selected!"
    for i in range(0,7):
        item = data['items'][i]['item']
        fname = data['items'][i]['forge_name']
        uses = data['items'][i]['uses']
        blessed = data['items'][i]['blessed']
        forged = data['items'][i]['forged']
        mt = data['items'][i]['mt']
        hit = data['items'][i]['hit']
        crit = data['items'][i]['crit']
        wt = data['items'][i]['wt']

        fname_off = characters[f'Item_{i+1}'][char]
        fstat_off = characters[f'Item_{i+1}_Forge'][char]

        if item:
            temp = f'04{characters[f'Item_{i+1}'][char][-6:]} {items['Offset'][item]}'
            output.append(temp)

        if uses:
            try:
                uses = int(uses)
                if uses > 255:
                    return f'Error: Uses for {item} is too high! Please enter a value between 0 and 255.'
                temp = f'00{characters[f'Item_{i+1}_Uses'][char][-6:]} 000000{hex(uses).replace('0x', '').zfill(2).upper()}'
                output.append(temp)
            except ValueError:
                return f'Error: Uses for {item} is not a number! Please enter a value between 0 and 255.'
        
        if blessed or forged or item:
            sts = 0
            if blessed:
                sts += int('10', 16)
            if forged:
                sts += int('20', 16)

                fnamecode = ''
                if len(fname) > 26:
                        return f'Error: Forge Name for {item} is too long! Please enter a name with 26 characters or less.'
                elif fname and len(fname) <= 26:
                    for c in fname:
                        fnamecode += format(ord(c), "x").zfill(2)
                else:
                    for c in item:
                        fnamecode += format(ord(c), "x").zfill(2)
                fnamecode = fnamecode.ljust(60, '0').upper()

                j = 0
                for k in range(0, 7):
                    if k == 0:
                        offset = hex(int(fname_off, 16) + 6).replace('0x', '').zfill(8).upper()
                        temp = f'02{offset[-6:]} 0000{fnamecode[:4]}'
                        if temp[-8:] != '00000000':
                            output.append(temp)
                    else:
                        offset = hex(int(fname_off, 16) + 8 + j).replace('0x', '').zfill(8).upper()
                        temp = f'04{offset[-6:]} {fnamecode[4+(j*2):12+(j*2)]}'
                        if temp[-8:] != '00000000':
                            output.append(temp)
                        j += 4
            
            equip = hex(sts).replace('0x', '').zfill(2).upper()
            temp = f'00{characters[f'Item_{i+1}_Status'][char][-6:]} 000000{equip}'
            output.append(temp)

        if mt or hit or wt or crit:
            if mt == '':
                mt = 0
            if hit == '':
                hit = 0
            if wt == True:
                wt = 'E0'
            else:
                wt = '00'
            if crit == '':
                crit = 0
            try:
                mt = int(mt)
                hit = int(hit)
                crit = int(crit)
                if mt > 255 or hit > 255 or crit > 255:
                    return f'Error: Stat for {item} is too high! Please enter a value between 0 and 255.'
                temp1 = f'02{fstat_off[-6:]} 0000{hex(mt).replace('0x', '').zfill(2).upper()}{hex(hit).replace('0x', '').zfill(2).upper()}'
                off2 = hex(int(fstat_off, 16) + 2).replace('0x', '').zfill(8).upper()
                temp2 = f'02{off2[-6:]} 0000{hex(crit).replace('0x', '').zfill(2).upper()}{wt}'
                output.append(temp1)
                output.append(temp2)
            except ValueError:
                return f'Error: Stat for {item} is not a number! Please enter a value between 0 and 255.'
    
    output.append('E0000000 80008000')
    if len(output) == 2:
        return "No changes made!"
    else:
        return "\n".join(output)

def get_class_code(data):
    output = []
    output.append('20B54158 8070F8BC')

    cls = data['class']
    if not cls:
        return "No class selected!"

    rank_names = [
        'Sword_Rank',
        'Lance_Rank',
        'Axe_Rank',
        'Bow_Rank',
        'Knife_Rank',
        'Strike_Rank',
        'Fire_Rank',
        'Thunder_Rank',
        'Wind_Rank',
        'Light_Rank',
        'Dark_Rank',
        'Staff_Rank'
    ]

    for i, name in enumerate(rank_names):
        if data['weapon_ranks'][i]:
            if data['weapon_ranks'][i] == 'SS':
                val = '014B'
            elif data['weapon_ranks'][i] == 'S':
                val = '00FB'
            elif data['weapon_ranks'][i] == 'A':
                val = '00B5'
            elif data['weapon_ranks'][i] == 'B':
                val = '0079'
            elif data['weapon_ranks'][i] == 'C':
                val = '0047'
            elif data['weapon_ranks'][i] == 'D':
                val = '001F'
            elif data['weapon_ranks'][i] == 'E':
                val = '0001'
            else:
                return f'Error: Invalid weapon rank for {name.replace('_', ' ')}! Please select a valid rank.'

            temp = f'02{classes[name][cls][-6:]} 0000{val}'
            output.append(temp)
    
    stat_names = [
        'Base_WT',
        'Base_Move',
        'Skill_Capacity',
        'Max_HP',
        'Max_STR',
        'Max_MAG',
        'Max_SKL',
        'Max_SP',
        'Max_LCK',
        'Max_DEF',
        'Max_RES'
    ]

    for i, name in enumerate(stat_names):
        if data['stats'][i]:
            try:
                num = int(data['stats'][i])
                if num > 255:
                    return f'Error: Stat for {name.replace('_', ' ')} is too high! Please enter a value between 0 and 255.'
                temp = f'00{classes[name][cls][-6:]} 000000{hex(num).replace("0x", "").zfill(2).upper()}'
                output.append(temp)
            except ValueError:
                return f'Error: Stat for {name} is not a number! Please enter a value between 0 and 255.'

    output.append('E0000000 80008000')
    if len(output) == 2:
        return "No changes made!"
    else:
        return "\n".join(output)

def get_item_code(data):
    output = []
    output.append('20B54158 8070F8BC')

    item = data['item']
    if not item:
        return "No item selected!"
    
    w_opts = ['Attack_Type', 'Weapon_Rank', 'EXP_Gain', 'Unlock', 'Char_Unlock', 'Infinite', 'Brave', 'Heal']
    d_opts = ['Attack Type', 'Weapon Rank', 'EXP Gain', 'Unlock', 'CharUnlock', 'Infinite', 'Brave', 'Heal']

    for w, d in zip(w_opts, d_opts):
        if data['misc_data'][d]:
            if d == 'Attack Type':
                if data['misc_data'][d] == 'STR':
                    val = '00'
                elif data['misc_data'][d] == 'MAG':
                    val = '06'
                else:
                    return f'Error: Invalid attack type for {item}! Please select a valid type.'
                temp = f'00{items[w][item][-6:]} 000000{val}'
                output.append(temp)
            elif d == 'Weapon Rank':
                if data['misc_data'][d] == 'SS':
                    val = '014B'
                elif data['misc_data'][d] == 'S':
                    val = '00FB'
                elif data['misc_data'][d] == 'A':
                    val = '00B5'
                elif data['misc_data'][d] == 'B':
                    val = '0079'
                elif data['misc_data'][d] == 'C':
                    val = '0047'
                elif data['misc_data'][d] == 'D':
                    val = '001F'
                elif data['misc_data'][d] == 'E':
                    val = '0001'
                else:
                    return f'Error: Invalid weapon rank for {item}! Please select a valid rank.'
                temp = f'02{items[w][item][-6:]} 0000{val}'
                output.append(temp)
            elif d == 'EXP Gain':
                temp = f'00{items[w][item][-6:]} 000000{hex(int(data["misc_data"][d])).replace("0x", "").zfill(2).upper()}'
                output.append(temp)
            elif d == 'Unlock':
                if data['misc_data'][d]:
                    val = '00'
                    temp = f'00{items[w][item][-6:]} 000000{val}'
                    output.append(temp)
            elif d in ['Infinite', 'Brave']:
                if data['misc_data'][d]:
                    val = '01'
                    temp = f'00{items[w][item][-6:]} 000000{val}'
                    output.append(temp)
            elif d == 'CharUnlock':
                if data['misc_data'][d]:
                    val = '0000'
                    temp = f'02{items[w][item][-6:]} 0000{val}'
                    output.append(temp)
            elif d == 'Heal':
                if data['misc_data'][d]:
                    val = '10'
                    temp = f'00{items[w][item][-6:]} 000000{val}'
                    output.append(temp)

    eq_bonus = ['HP_Increase', 'STR_Increase', 'MAG_Increase', 'SKL_Increase', 'SP_Increase', 'LCK_Increase', 'DEF_Increase', 'RES_Increase', 'Move_Increase', 'CN-WT_Increase']

    for i, bonus in enumerate(eq_bonus):
        if data['equip_bonuses'][i]:
            try:
                num = int(data['equip_bonuses'][i])
                if num > 255:
                    return f'Error: Equip Bonus for {bonus.replace("_", " ")} is too high! Please enter a value between 0 and 255.'
                temp = f'00{items[bonus][item][-6:]} 000000{hex(int(data["equip_bonuses"][i])).replace("0x", "").zfill(2).upper()}'
                output.append(temp)
            except ValueError:
                return f'Error: Equip Bonus for {bonus} is not a number! Please enter a value between 0 and 255.'

    output.append('E0000000 80008000')
    if len(output) == 2:
        return "No changes made!"
    else:
        return "\n".join(output)

def get_keybind_code(data):
    val = 0
    if data['controller'] == '':
        return '20B54158 8070F8BC'
    elif data['controller'] == 'Wiimote+Nunchuck':
        # Left
        if data['keys'][0]:
            val += int('1', 16)
        # Right
        if data['keys'][1]:
            val += int('2', 16)
        # Up
        if data['keys'][2]:
            val += int('8', 16)
        # Down
        if data['keys'][3]:
            val += int('4', 16)
        # A
        if data['keys'][4]:
            val += int('800', 16)
        # B
        if data['keys'][5]:
            val += int('400', 16)
        # C
        if data['keys'][6]:
            val += int('4000', 16)
        # Z
        if data['keys'][7]:
            val += int('2000', 16)
        # 1
        if data['keys'][8]:
            val += int('200', 16)
        # 2
        if data['keys'][9]:
            val += int('100', 16)
        # Plus
        if data['keys'][10]:
            val += int('10', 16)
        # Minus
        if data['keys'][11]:
            val += int('1000', 16)
        return f'28 {hex(val).replace('0x', '').zfill(8)}'

    elif data['controller'] == 'Classic Controller':
        val = 0
        # Left
        if data['keys'][0]:
            val += int('2', 16)
        # Right
        if data['keys'][1]:
            val += int('8000', 16)
        # Up
        if data['keys'][2]:
            val += int('1', 16)
        # Down
        if data['keys'][3]:
            val += int('4000', 16)
        # A
        if data['keys'][4]:
            val += int('10', 16)
        # B
        if data['keys'][5]:
            val += int('40', 16)
        # X
        if data['keys'][6]:
            val += int('8', 16)
        # Y
        if data['keys'][7]:
            val += int('20', 16)
        # ZL
        if data['keys'][8]:
            val += int('80', 16)
        # ZR
        if data['keys'][9]:
            val += int('4', 16)
        # L
        if data['keys'][10]:
            val += int('2000', 16)
        # R
        if data['keys'][11]:
            val += int('200', 16)
        # Plus
        if data['keys'][12]:
            val += int('400', 16)
        # Minus
        if data['keys'][13]:
            val += int('1000', 16)
        return f'283D79BA {hex(val).replace('0x', '').zfill(8)}'
    
    elif data['controller'] == 'GameCube Controller':
        # Left
        if data['keys'][0]:
            val += int('1', 16)
        # Right
        if data['keys'][1]:
            val += int('2', 16)
        # Up
        if data['keys'][2]:
            val += int('8', 16)
        # Down
        if data['keys'][3]:
            val += int('4', 16)
        # A
        if data['keys'][4]:
            val += int('100', 16)
        # B
        if data['keys'][5]:
            val += int('200', 16)
        # X
        if data['keys'][6]:
            val += int('400', 16)
        # Y
        if data['keys'][7]:
            val += int('800', 16)
        # Z
        if data['keys'][8]:
            val += int('10', 16)
        # L
        if data['keys'][9]:
            val += int('40', 16)
        # R
        if data['keys'][10]:
            val += int('20', 16)
        # Start
        if data['keys'][11]:
            val += int('1000', 16)
        return f'283D7928 {hex(val).replace('0x', '').zfill(8)}'

In [11]:
# GUI

class CodeGeneratorGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("FE:RD Code Creator")

        # Set dark mode colors
        style = ttk.Style()
        style.theme_use('clam')
        style.configure('TNotebook', background='#000000', foreground='#ffffff')
        style.configure('TNotebook.Tab', background='#000000', foreground='#ffffff')
        style.map('TNotebook.Tab', background=[('selected', '#2e2e2e')])
        
        # Configure text color for various widgets
        style.configure('TFrame', background='#2e2e2e')
        style.configure('TLabel', background='#2e2e2e', foreground='#ffffff')
        style.configure('TEntry', fieldbackground='#ffffff', foreground='#000000')
        style.configure('TCombobox', fieldbackground='#ffffff', foreground='#000000')
        style.map('TCombobox', fieldbackground=[('readonly', '#ffffff')], foreground=[('readonly', '#000000')])
        style.configure('TCheckbutton', background='#2e2e2e', foreground='#000000')

        self.notebook = ttk.Notebook(root)
        self.notebook.pack(fill="both", expand=True)

        self.info_tab()
        self.keybinds_tab()
        self.character_tab()
        self.class_tab()
        self.items_tab()

    def copy_to_clipboard(self, text):
        self.root.clipboard_clear()  # Clear the clipboard
        self.root.clipboard_append(text)  # Append the text to the clipboard

    def output_code(self, code):
        # Create a new window for the message box
        message_window = tk.Toplevel(self.root)
        message_window.title("Code")
        message_window.configure(bg='#2e2e2e')
        
        # Add a label to display the output
        output_label = ttk.Label(message_window, text=code, justify="left")
        output_label.pack(padx=10, pady=10)

        if code not in ["No character selected!", "No class selected!", "No item selected!", "No changes made!"] and 'Error:' not in code:
            # Add a button to copy to clipboard
            copy_button = ttk.Button(message_window, text="Copy to Clipboard", command=lambda: self.copy_to_clipboard(code))
            copy_button.pack(pady=5)

    def info_tab(self):
        info_tab = ttk.Frame(self.notebook)
        self.notebook.add(info_tab, text="Info")

        desc_frame = ttk.Frame(info_tab)
        desc_frame.pack(fill="both", expand=True)

        i = 3
        for tab in list(desc):
            if tab == "intro":
                label = ttk.Label(desc_frame, text=desc[tab], wraplength=950, justify="center")
                label.grid(row=0, column=0, pady=10, columnspan=3)
                ttk.Separator(desc_frame, orient="horizontal").grid(row=1, column=0, columnspan=3, sticky="ew")
            else:
                label = ttk.Label(desc_frame, text=tab, wraplength=150)
                label.grid(row=i, column=0, pady=10, sticky="w")
                ttk.Separator(desc_frame, orient="vertical").grid(row=i, column=1, padx=5, rowspan=1, sticky="ns")
                label = ttk.Label(desc_frame, text=desc[tab], wraplength=790, justify="left")
                label.grid(row=i, column=2, pady=10, sticky="w")
                ttk.Separator(desc_frame, orient="horizontal").grid(row=i+1, column=0, columnspan=3, sticky="ew")
            i += 2

    def keybinds_tab(self):
        key_tab = ttk.Frame(self.notebook)
        self.notebook.add(key_tab, text="Keybind Activation")

        self.keybinds = {
            "": "",
            # "Wiimote+Nunchuck": [
            #     "Left",
            #     "Right",
            #     "Up",
            #     "Down",
            #     "A",
            #     "B",
            #     "C",
            #     "Z",
            #     "1",
            #     "2",
            #     "+",
            #     "-"
                
            # ],
            'Classic Controller': [
                "Left",
                "Right",
                "Up",
                "Down",
                "A",
                "B",
                "X",
                "Y",
                "ZL",
                "ZR",
                "L",
                "R",
                "+",
                "-"
            ],
            'GameCube Controller': [
                "Left",
                "Right",
                "Up",
                "Down",
                "A",
                "B",
                "X",
                "Y",
                "Z",
                "L",
                "R",
                "Start"
            ]
        }

        controller_frame = ttk.Frame(key_tab)
        controller_frame.grid(row=0, column=0, pady=10, sticky="nsew")

        # Dropdown Section
        ttk.Label(controller_frame, text="Controller:").grid(row=0, column=0, padx=5)
        self.dropdown = ttk.Combobox(controller_frame, values=list(self.keybinds.keys()))
        self.dropdown.grid(row=0, column=1)
        self.dropdown.bind("<<ComboboxSelected>>", self.update_checkboxes)

        # Separator
        ttk.Separator(key_tab, orient="horizontal").grid(row=1, column=0, columnspan=2, sticky="ew")

        # Checkboxes Section
        self.checkboxes = []
        self.checkbox_frame = ttk.Frame(key_tab)
        self.checkbox_frame.grid(row=2, column=0, sticky="nsew")

    def update_checkboxes(self, event):
        for checkbox in self.checkbox_frame.winfo_children():
            checkbox.destroy()
        self.checkboxes = []
        
        selected_option = self.dropdown.get()
        if selected_option == "":
            pass
        elif selected_option in self.keybinds:
            ttk.Label(self.checkbox_frame, text="Buttons").grid(row=0, column=0, rowspan=len(self.keybinds[selected_option]), padx=10)
            ttk.Separator(self.checkbox_frame, orient="vertical").grid(row=0, column=1, rowspan=len(self.keybinds[selected_option]), sticky="ns", pady=5)
            for i, option in enumerate(self.keybinds[selected_option]):
                ttk.Label(self.checkbox_frame, text=option).grid(row=i, column=3, padx=5, sticky="e")
                var = tk.BooleanVar()
                checkbox = ttk.Checkbutton(self.checkbox_frame, variable=var)
                checkbox.grid(row=i, column=4)
                self.checkboxes.append(var)

    def character_tab(self):
        char_tab = ttk.Frame(self.notebook)
        self.notebook.add(char_tab, text="Character")

        #region Row 0

        # Character Select Section
        char_frame = ttk.Frame(char_tab)
        char_frame.grid(row=0, column=0, pady=10, sticky="nsew")
        
        ttk.Label(char_frame, text="Character Select:").grid(row=0, column=0, padx=5)
        self.character_select = ttk.Combobox(char_frame, values=char_sel)
        self.character_select.grid(row=0, column=1)

        #endregion

        # Row 1 Separator
        ttk.Separator(char_tab, orient="horizontal").grid(row=1, column=0, columnspan=2, sticky="ew")

        #region Row 2

        # Items Section
        self.item_entries = []
        self.item_table = ttk.Frame(char_tab)
        self.item_table.grid(row=2, column=0)

        headers = ["Items", "Forge Name", "Uses", "Blessed", "Forged", "Might", "Hit", "Crit", "Weightless"]
        for i, header in enumerate(headers):
            ttk.Label(self.item_table, text=header).grid(row=0, column=i)

        for row in range(1, 8):
            item_row = []
            item_combobox = ttk.Combobox(self.item_table, values=item_sel, width=25)
            item_combobox.grid(row=row, column=0, padx=1, pady=1)
            item_row.append(item_combobox)

            for col in range(1, 9):
                if col == 3 or col == 4 or col == 8:
                    var = tk.BooleanVar(value=False)
                    if col == 3:
                        blessed_checkbox = ttk.Checkbutton(self.item_table, text="", variable=var)
                        blessed_checkbox.grid(row=row, column=col)
                    elif col == 4:
                        forged_checkbox = ttk.Checkbutton(self.item_table, text="", variable=var)
                        forged_checkbox.grid(row=row, column=col)
                    elif col == 8:
                        wt_checkbox = ttk.Checkbutton(self.item_table, text="", variable=var)
                        wt_checkbox.grid(row=row, column=col)
                    item_row.append(var)
                else:
                    if col == 1:
                        entry = ttk.Entry(self.item_table, width=20)
                        entry.grid(row=row, column=col, padx=1)
                        item_row.append(entry)
                    else:
                        entry = ttk.Entry(self.item_table, width=5)
                        entry.grid(row=row, column=col, padx=1)
                        item_row.append(entry)

            self.item_entries.append(item_row)

        #endregion

        # Row 3 Separator
        ttk.Separator(char_tab, orient="horizontal").grid(row=3, column=0, columnspan=2, sticky="ew")

        # Row 4 Generate Button
        generate_button = ttk.Button(char_tab, text="Generate", command=self.generate_character)
        generate_button.grid(row=4, column=0, sticky="nsew")

    def generate_character(self):
        character_data = {
            "character": self.character_select.get(),
            "items": [
                {
                    "item": row[0].get(),
                    "forge_name": row[1].get(),
                    "uses": row[2].get(),
                    "blessed": row[3].get(),
                    "forged": row[4].get(),
                    "mt": row[5].get(),
                    "hit": row[6].get(),
                    "crit": row[7].get(),
                    "wt": row[8].get(),
                } for row in self.item_entries
            ]
        }

        keybinds_data = {
            "controller": self.dropdown.get(),
            "keys": [key.get() for key in self.checkboxes]
        }
        
        key_code = get_keybind_code(keybinds_data)
        output = get_char_code(character_data, key_code)
        self.output_code(output)

    def class_tab(self):
        class_tab = ttk.Frame(self.notebook)
        self.notebook.add(class_tab, text="Class")

        #region Row 0

        # Class Select Section
        class_frame = ttk.Frame(class_tab)
        class_frame.grid(row=0, column=0, pady=10, sticky="nsew")
        
        ttk.Label(class_frame, text="Class Select:").grid(row=0, column=0, padx=5)
        self.class_select = ttk.Combobox(class_frame, values=class_sel, width=40)
        self.class_select.grid(row=0, column=1)

        #endregion

        # Row 1 Separator
        ttk.Separator(class_tab, orient="horizontal").grid(row=1, column=0, columnspan=1, sticky="ew")

        #region Row 2

        # Create frame for weapon ranks and stats
        ranks_stats = ttk.Frame(class_tab)
        ranks_stats.grid(row=2, column=0)

        # Weapon Ranks Section
        ttk.Label(ranks_stats, text="Weapon Ranks").grid(row=0, column=0, pady=5, columnspan=2)
        weapon_ranks = ["Sword Rank", "Lance Rank", "Axe Rank", "Bow Rank", "Knife Rank", "Strike Rank",
                        "Fire Rank", "Thunder Rank", "Wind Rank", "Light Rank", "Dark Rank", "Staff Rank"]
        self.weapon_rank_comboboxes = []

        for i, rank in enumerate(weapon_ranks):
            ttk.Label(ranks_stats, text=rank).grid(row=i+1, column=0, padx=5, sticky="e")
            combobox = ttk.Combobox(ranks_stats, values=["SS", "S", "A", "B", "C", "D", "E"], width=5)
            combobox.grid(row=i+1, column=1, padx=5, pady=1)
            self.weapon_rank_comboboxes.append(combobox)
        
        # Separator
        ttk.Separator(ranks_stats, orient="vertical").grid(row=0, column=2, rowspan=13, sticky="ns")

        # Stats Section
        ttk.Label(ranks_stats, text="Stats").grid(row=0, column=3, pady=5, columnspan=2)
        stats = ["Base WT", "Base Move", "Skill Capacity", "Max HP", "Max STR", "Max MAG", "Max SKL",
                "Max SP", "Max LCK", "Max DEF", "Max RES"]
        self.stats_entries = []

        for i, stat in enumerate(stats):
            ttk.Label(ranks_stats, text=stat).grid(row=i+1, column=3, padx=5, sticky="e")
            entry = ttk.Entry(ranks_stats, width=5)
            entry.grid(row=i+1, column=4, pady=1)
            self.stats_entries.append(entry)

        #endregion

        # Row 3 Separator
        ttk.Separator(class_tab, orient="horizontal").grid(row=3, column=0, columnspan=1, sticky="ew")

        # Row 4 Generate Button
        generate_button = ttk.Button(class_tab, text="Generate", command=self.generate_class)
        generate_button.grid(row=4, column=0, sticky="nsew")

    def generate_class(self):
        class_data = {
            "class": self.class_select.get(),
            "weapon_ranks": [cb.get() for cb in self.weapon_rank_comboboxes],
            "stats": [entry.get() for entry in self.stats_entries]
        }

        output = get_class_code(class_data)

        self.output_code(output)

    def items_tab(self):
        items_tab = ttk.Frame(self.notebook)
        self.notebook.add(items_tab, text="Items")

        #region Row 0

        # Item Select Section
        item_frame = ttk.Frame(items_tab)
        item_frame.grid(row=0, column=0, pady=10, sticky="nsew")
        
        ttk.Label(item_frame, text="Item Select:").grid(row=0, column=0, padx=5)
        self.item_select = ttk.Combobox(item_frame, values=item_sel, width=25)
        self.item_select.grid(row=0, column=1)

        #endregion

        # Row 1 Separator
        ttk.Separator(items_tab, orient="horizontal").grid(row=1, column=0, columnspan=5, sticky="ew")

        #region Row 2

        data_bonus = ttk.Frame(items_tab)
        data_bonus.grid(row=2, column=0)

        # Misc Weapon Data Section
        ttk.Label(data_bonus, text="Misc Weapon Data").grid(row=0, column=0, columnspan=2, pady=5)
        misc_data_labels = ["Attack Type", "Weapon Rank", "EXP Gain", "Unlock", "CharUnlock", "Infinite", "Brave", "Heal"]
        self.misc_data = {}

        for i, label in enumerate(misc_data_labels):
            ttk.Label(data_bonus, text=label).grid(row=i+1, column=0, padx=5, sticky="e")
            if label == "Attack Type":
                self.misc_data[label] = ttk.Combobox(data_bonus, values=["STR", "MAG"], width=5)
            elif label == "Weapon Rank":
                self.misc_data[label] = ttk.Combobox(data_bonus, values=["SS", "S", "A", "B", "C", "D", "E"], width=5)
            elif label in ["Unlock", "CharUnlock", "Infinite", "Brave", "Heal"]:
                self.misc_data[label] = tk.BooleanVar()
                ttk.Checkbutton(data_bonus, variable=self.misc_data[label]).grid(row=i+1, column=1)
                continue
            else:
                self.misc_data[label] = ttk.Entry(data_bonus, width=5)

            self.misc_data[label].grid(row=i+1, column=1, padx=5, pady=1)

        # Separator
        ttk.Separator(data_bonus, orient="vertical").grid(row=0, column=2, rowspan=11, sticky="ns", pady=5)

        # Equip Bonuses Section
        ttk.Label(data_bonus, text="Equip Bonuses").grid(row=0, column=3, columnspan=2, pady=5)
        equip_bonus_labels = ["HP Increase", "STR Increase", "MAG Increase", "SKL Increase", "SP Increase",
                                "LCK Increase", "DEF Increase", "RES Increase", "Move Increase", "CN/WT Increase"]
        self.equip_bonus_entries = []

        for i, label in enumerate(equip_bonus_labels):
            ttk.Label(data_bonus, text=label).grid(row=i+1, column=3, padx=5, sticky="e")
            entry = ttk.Entry(data_bonus, width=5)
            entry.grid(row=i+1, column=4, padx=0, pady=1)
            self.equip_bonus_entries.append(entry)

        #endregion

        # Row 3 Separator
        ttk.Separator(items_tab, orient="horizontal").grid(row=3, column=0, columnspan=5, sticky="ew")

        # Row 4 Generate Button
        generate_button = ttk.Button(items_tab, text="Generate", command=self.generate_item)
        generate_button.grid(row=4, column=0, sticky="nsew")

    def generate_item(self):
        item_data = {
            "item": self.item_select.get(),
            "misc_data": {key: (var.get() if isinstance(var, tk.BooleanVar) else var.get())
                        for key, var in self.misc_data.items()},
            "equip_bonuses": [entry.get() for entry in self.equip_bonus_entries]
        }

        output = get_item_code(item_data)  # Get the output from get_char_code

        self.output_code(output)

    def other_tab(self):
        pass

In [12]:
# Main

if __name__ == "__main__":
    root = tk.Tk()
    app = CodeGeneratorGUI(root)
    root.mainloop()